In [4]:
import os
from pathlib import Path

from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy_utils import database_exists, create_database
import pandas as pd

## Functions

In [5]:
def get_credentials():
    path = Path.cwd() / 'config' / '.ENV'
    load_dotenv(path)
    user = os.getenv('POSTGRESUSER')
    password = os.getenv('PASSWORD')
    host = os.getenv('HOST')
    port = os.getenv('PORT')
    dbname = os.getenv('DBNAME')
    credentials = {
        'user':user,
        'password':password,
        'host':host,
        'port':port,
        'dbname':dbname,
    }
    return credentials


def get_engine(user, password, host, port, dbname):
    url = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'
    if not database_exists(url):
        create_database(url)
    engine = create_engine(url, pool_size=50, echo=False)
    return engine

def get_postgres_engine():
    keys = get_credentials()
    engine = get_engine(keys['user'],keys['password'], keys['host'],keys['port'],keys['dbname'])
    return engine

# **CRUD**

In [4]:
from sqlalchemy import Column, Integer, MetaData, String, Table, create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

from sqlalchemy.orm import sessionmaker

In [6]:
# Declaração de uma tabela
engine = get_postgres_engine()
Base = declarative_base()

class User(Base):
    __tablename__ = 'Users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)

Base.metadata.create_all(engine)

## Create

In [7]:
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
user = User(name='Joel', age=20)
user_2 = User(name='Rool', age=30)
user_3 = User(name='Melo', age=40)
user_4 = User(name='Junior', age=50)

session.add(user)
session.add_all([user_2, user_3, user_4])
session.commit()

## Read

In [9]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(engine)
session = Session()

users = session.query(User).all()
user = users[0]

print(user.id)
print(user.name)
print(user.age)

2
Rool
30


In [10]:
users = session.query(User).all()
for user in users:
    print(f'User id: {user.id}, User name: {user.name}, User age: {user.age}')

User id: 2, User name: Rool, User age: 30
User id: 3, User name: Melo, User age: 40
User id: 4, User name: Junior, User age: 50
User id: 5, User name: Joel, User age: 20
User id: 6, User name: Rool, User age: 30
User id: 7, User name: Melo, User age: 40
User id: 8, User name: Junior, User age: 50
User id: 9, User name: Joel, User age: 20
User id: 10, User name: Rool, User age: 30
User id: 11, User name: Melo, User age: 40
User id: 12, User name: Junior, User age: 50


In [11]:
user = session.query(User).filter_by(id=2).one_or_none()
print(user.id)
print(user.name)
print(user.age)

2
Rool
30


## Update

In [13]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(engine)
session = Session()

user = session.query(User).filter_by(id=2).one_or_none()

print(user.name)
user.name = "Outro nome"
print(user.name)

session.commit()

Rool
Outro nome


## Delete

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(engine)
session = Session()

user = session.query(User).filter_by(id=2).one_or_none()
session.delete(user)
session.commit()

# **Ordenando os Dados**

In [22]:
import random
from sqlalchemy.orm import sessionmaker

engine = get_postgres_engine()
Session = sessionmaker(engine)
session = Session()

### Resetando os dados da tabela

In [36]:
session.query(User).delete()
session.commit()

### Colocando dados falsos no banco  de dados

In [37]:
names = ['Arthur','Rool','Ramos','Melo','Anna','Victoria']
ages = [25,26,26,67,69,42,36,58,91,46,72]


for x in range(20):
    user = User(name=random.choice(names), age=random.choice(ages))
    session.add(user)
session.commit()

### Por idade do maior para o menor (decrecente)

In [48]:
users = session.query(User).order_by(User.age.desc()).all()

for user in users:
    print(user.id, user.name, user.age)

131 Anna 91
118 Melo 91
124 Rool 72
132 Rool 72
130 Melo 69
126 Rool 69
113 Melo 58
120 Arthur 46
127 Arthur 46
115 Ramos 42
119 Victoria 42
116 Anna 36
121 Rool 36
114 Victoria 26
122 Ramos 26
123 Ramos 26
125 Arthur 26
129 Anna 26
128 Ramos 25
117 Victoria 25


### Por nome do menor para o maior (crecente)

In [50]:
users = session.query(User).order_by(User.name.asc()).all()

for user in users:
    print(user.id, user.name, user.age)

116 Anna 36
129 Anna 26
131 Anna 91
127 Arthur 46
125 Arthur 26
120 Arthur 46
130 Melo 69
113 Melo 58
118 Melo 91
123 Ramos 26
115 Ramos 42
122 Ramos 26
128 Ramos 25
121 Rool 36
132 Rool 72
124 Rool 72
126 Rool 69
119 Victoria 42
117 Victoria 25
114 Victoria 26


# **Filtrando os Dados**

In [6]:
from sqlalchemy.orm import sessionmaker

engine = get_postgres_engine()
Session = sessionmaker(engine)
session = Session()